# 04. Inverted Index - Efficient IR Data Structure

## Table of Contents
1. [Introduction](#introduction)
2. [Theory: Inverted Index](#theory)
3. [Building the Index](#building)
4. [Query Processing with Index](#queries)
5. [Performance Comparison](#performance)
6. [Summary](#summary)

---

## 1. Introduction <a name="introduction"></a>

The **Inverted Index** is the fundamental data structure for modern search engines. It enables fast retrieval by mapping terms to the documents that contain them.

### Why Inverted Index?
- **Speed**: O(1) lookup for term → documents mapping
- **Space Efficient**: Stores only non-zero entries
- **Scalable**: Used by Google, Elasticsearch, Apache Lucene

---

## 2. Theory: Inverted Index <a name="theory"></a>

### Forward Index vs. Inverted Index

**Forward Index** (Document → Terms):
```
doc01: [नेपाल, इतिहास, संस्कृति, ...]
doc02: [नेपाल, हिमाल, पर्यटक, ...]
doc03: [शिक्षा, प्रविधि, नेपाल, ...]
```
❌ **Problem**: To find docs with "नेपाल", must scan ALL documents

**Inverted Index** (Term → Documents):
```
नेपाल: [doc01, doc02, doc03, doc05, ...]
हिमाल: [doc02, doc09]
शिक्षा: [doc03]
```
✓ **Solution**: Direct lookup! O(1) time to find documents

### Index Structure:

1. **Dictionary (Lexicon)**
   - All unique terms in collection
   - Sorted alphabetically
   - Points to posting list

2. **Postings List**
   - For each term: list of documents containing it
   - Can include: position, frequency, metadata

```
Dictionary        Postings Lists
----------        --------------
अर्थतन्त्र   →    [doc04]
नेपाल       →    [doc01, doc02, doc03, doc04, doc05, ...]
पर्यटक      →    [doc02]
शिक्षा      →    [doc03]
```

### Real-World Usage:
- **Google**: Uses distributed inverted indices
- **Elasticsearch**: JSON-based inverted index
- **Apache Lucene**: Open-source inverted index library

---

## 3. Building the Index <a name="building"></a>

In [1]:
from pathlib import Path
from collections import defaultdict

# Load and preprocess documents (same as previous notebooks)
DATA_DIR = Path('../data')

def load_documents(data_dir):
    documents = {}
    for file_path in sorted(data_dir.glob('doc*.txt')):
        with open(file_path, 'r', encoding='utf-8') as f:
            documents[file_path.stem] = f.read()
    return documents

def load_stopwords(file_path):
    stopwords = set()
    with open(file_path, 'r', encoding='utf-8') as f:
        next(f)
        for line in f:
            stopwords.add(line.strip())
    return stopwords

def load_stemming_dict(file_path):
    stem_dict = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        next(f)
        for line in f:
            parts = line.strip().split(',')
            if len(parts) == 2:
                stem_dict[parts[0]] = parts[1]
    return stem_dict

def tokenize(text):
    tokens = text.split()
    cleaned = []
    for token in tokens:
        token = token.strip('।,.!?;:"\'-()[]{}/')
        if token and any('\u0900' <= c <= '\u097F' for c in token):
            cleaned.append(token)
    return cleaned

def preprocess_text(text, stopwords, stem_dict):
    tokens = tokenize(text)
    tokens = [t for t in tokens if t not in stopwords]
    tokens = [stem_dict.get(t, t) for t in tokens]
    return tokens

documents = load_documents(DATA_DIR)
stopwords = load_stopwords(DATA_DIR / 'nepali_stopwords.csv')
stem_dict = load_stemming_dict(DATA_DIR / 'nepali_stemming.csv')

preprocessed_docs = {}
for doc_id, text in documents.items():
    preprocessed_docs[doc_id] = preprocess_text(text, stopwords, stem_dict)

print(f"✓ Loaded {len(preprocessed_docs)} documents")

✓ Loaded 10 documents


In [2]:
def build_inverted_index(preprocessed_docs):
    """
    Build an inverted index from preprocessed documents.
    
    For each term, store the list of documents containing it.
    This is a basic boolean inverted index.
    
    Parameters:
    -----------
    preprocessed_docs : dict
        Mapping from doc_id to list of preprocessed terms
    
    Returns:
    --------
    dict : Inverted index mapping term → set of document IDs
    """
    inverted_index = defaultdict(set)
    
    # For each document
    for doc_id, terms in preprocessed_docs.items():
        # For each unique term in document
        unique_terms = set(terms)
        for term in unique_terms:
            # Add document to term's posting list
            inverted_index[term].add(doc_id)
    
    # Convert defaultdict to regular dict for clarity
    return dict(inverted_index)

# Build the inverted index
inverted_index = build_inverted_index(preprocessed_docs)

print(f"✓ Built inverted index")
print(f"  Unique terms: {len(inverted_index)}")
print(f"  Total postings: {sum(len(postings) for postings in inverted_index.values())}")

✓ Built inverted index
  Unique terms: 398
  Total postings: 474


In [3]:
# Examine the index structure
def show_index_sample(index, sample_terms):
    """
    Display sample entries from the inverted index.
    """
    print("\n📚 Inverted Index Sample:")
    print("="*70)
    print(f"{'Term':<20} {'Document IDs':<40} {'Frequency'}")
    print("="*70)
    
    for term in sample_terms:
        if term in index:
            postings = sorted(index[term])
            doc_str = ', '.join(postings)
            if len(doc_str) > 38:
                doc_str = doc_str[:35] + '...'
            print(f"{term:<20} {doc_str:<40} {len(postings)}")
        else:
            print(f"{term:<20} {'(not found)':<40} 0")
    
    print("="*70)

# Show interesting terms
interesting_terms = ['नेपाल', 'हिमाल', 'शिक्षा', 'पर्यटक', 'स्वास्थ्य', 'कृषि', 'राजनीति', 'संस्कृति']
show_index_sample(inverted_index, interesting_terms)


📚 Inverted Index Sample:
Term                 Document IDs                             Frequency
नेपाल                doc01, doc02, doc03, doc04, doc05, ...   10
हिमाल                doc01, doc02, doc09                      3
शिक्षा               doc03                                    1
पर्यटक               doc02                                    1
स्वास्थ्य            doc07                                    1
कृषि                 doc04, doc09                             2
राजनीति              doc10                                    1
संस्कृति             doc01, doc06                             2


In [4]:
# Statistics about the index
def analyze_index(index):
    """
    Compute statistics about the inverted index.
    """
    posting_lengths = [len(postings) for postings in index.values()]
    
    stats = {
        'total_terms': len(index),
        'total_postings': sum(posting_lengths),
        'avg_postings_per_term': round(sum(posting_lengths) / len(index), 2),
        'max_postings': max(posting_lengths),
        'min_postings': min(posting_lengths)
    }
    
    # Find most common terms
    term_freq = [(term, len(postings)) for term, postings in index.items()]
    term_freq.sort(key=lambda x: x[1], reverse=True)
    
    return stats, term_freq[:10]

stats, top_terms = analyze_index(inverted_index)

print("\n📊 Index Statistics:")
print("="*70)
for key, value in stats.items():
    print(f"  {key.replace('_', ' ').title()}: {value}")

print("\n📈 Top 10 Most Frequent Terms:")
print("="*70)
for i, (term, freq) in enumerate(top_terms, 1):
    print(f"  {i}. {term:<20} appears in {freq} documents")
print("="*70)


📊 Index Statistics:
  Total Terms: 398
  Total Postings: 474
  Avg Postings Per Term: 1.19
  Max Postings: 10
  Min Postings: 1

📈 Top 10 Most Frequent Terms:
  1. नेपाल                appears in 10 documents
  2. महत्वपूर्ण           appears in 5 documents
  3. विकास                appears in 4 documents
  4. राष्ट्रिय            appears in 4 documents
  5. पुरानो               appears in 3 documents
  6. हिमाल                appears in 3 documents
  7. पोखरा                appears in 3 documents
  8. सबैभन्दा             appears in 3 documents
  9. क्षेत्रमा            appears in 3 documents
  10. सरकारले              appears in 3 documents


---

## 4. Query Processing with Index <a name="queries"></a>

Using the inverted index makes query processing much more efficient!

In [5]:
def search_single_term(term, index):
    """
    Search for documents containing a term.
    
    With inverted index: O(1) lookup!
    """
    return index.get(term, set())

def search_and(term1, term2, index):
    """
    Boolean AND using inverted index.
    
    Returns documents containing BOTH terms.
    """
    docs1 = search_single_term(term1, index)
    docs2 = search_single_term(term2, index)
    return docs1 & docs2

def search_or(term1, term2, index):
    """
    Boolean OR using inverted index.
    
    Returns documents containing EITHER term.
    """
    docs1 = search_single_term(term1, index)
    docs2 = search_single_term(term2, index)
    return docs1 | docs2

def search_not(term1, term2, index, all_docs):
    """
    Boolean NOT using inverted index.
    
    Returns documents with term1 but NOT term2.
    """
    docs1 = search_single_term(term1, index)
    docs2 = search_single_term(term2, index)
    return docs1 - docs2

print("✓ Query functions defined")

✓ Query functions defined


In [6]:
# Test queries
print("\n🔍 Query Examples:\n")

# Query 1
print("1. Single term: 'नेपाल'")
results = search_single_term('नेपाल', inverted_index)
print(f"   Results: {sorted(results)}")
print(f"   Count: {len(results)} documents\n")

# Query 2
print("2. AND query: 'नेपाल' AND 'हिमाल'")
results = search_and('नेपाल', 'हिमाल', inverted_index)
print(f"   Results: {sorted(results)}")
print(f"   Count: {len(results)} documents\n")

# Query 3
print("3. OR query: 'शिक्षा' OR 'स्वास्थ्य'")
results = search_or('शिक्षा', 'स्वास्थ्य', inverted_index)
print(f"   Results: {sorted(results)}")
print(f"   Count: {len(results)} documents\n")

# Query 4
all_doc_ids = set(preprocessed_docs.keys())
print("4. NOT query: 'नेपाल' AND NOT 'पर्यटक'")
results = search_not('नेपाल', 'पर्यटक', inverted_index, all_doc_ids)
print(f"   Results: {sorted(results)}")
print(f"   Count: {len(results)} documents")


🔍 Query Examples:

1. Single term: 'नेपाल'
   Results: ['doc01', 'doc02', 'doc03', 'doc04', 'doc05', 'doc06', 'doc07', 'doc08', 'doc09', 'doc10']
   Count: 10 documents

2. AND query: 'नेपाल' AND 'हिमाल'
   Results: ['doc01', 'doc02', 'doc09']
   Count: 3 documents

3. OR query: 'शिक्षा' OR 'स्वास्थ्य'
   Results: ['doc03', 'doc07']
   Count: 2 documents

4. NOT query: 'नेपाल' AND NOT 'पर्यटक'
   Results: ['doc01', 'doc03', 'doc04', 'doc05', 'doc06', 'doc07', 'doc08', 'doc09', 'doc10']
   Count: 9 documents


---

## 5. Performance Comparison <a name="performance"></a>

Let's compare inverted index vs. scanning all documents.

In [7]:
import time

def search_by_scanning(term, preprocessed_docs):
    """
    Naive approach: Scan all documents to find term.
    O(n) where n = number of documents
    """
    results = set()
    for doc_id, terms in preprocessed_docs.items():
        if term in terms:
            results.add(doc_id)
    return results

# Compare performance
test_term = 'नेपाल'
num_trials = 1000

# Method 1: Scanning
start = time.time()
for _ in range(num_trials):
    results_scan = search_by_scanning(test_term, preprocessed_docs)
time_scan = time.time() - start

# Method 2: Inverted Index
start = time.time()
for _ in range(num_trials):
    results_index = search_single_term(test_term, inverted_index)
time_index = time.time() - start

print("\n⚡ Performance Comparison:")
print("="*70)
print(f"Number of trials: {num_trials}")
print(f"Search term: '{test_term}'")
print(f"\nScanning all documents:")
print(f"  Total time: {time_scan*1000:.2f} ms")
print(f"  Average per query: {time_scan/num_trials*1000:.4f} ms")
print(f"\nInverted Index:")
print(f"  Total time: {time_index*1000:.2f} ms")
print(f"  Average per query: {time_index/num_trials*1000:.4f} ms")
print(f"\nSpeedup: {time_scan/time_index:.1f}x faster!")
print("="*70)

print("\n💡 Note: With 10 documents, the difference is small.")
print("   With millions of documents (like Google), inverted index is ESSENTIAL!")


⚡ Performance Comparison:
Number of trials: 1000
Search term: 'नेपाल'

Scanning all documents:
  Total time: 1.52 ms
  Average per query: 0.0015 ms

Inverted Index:
  Total time: 0.00 ms
  Average per query: 0.0000 ms


ZeroDivisionError: float division by zero

---

## 6. Summary <a name="summary"></a>

### What We Learned:

1. **Inverted Index Structure**
   - Maps terms → documents (reverse of forward index)
   - Dictionary + Posting lists
   - Foundation of modern search engines

2. **Building the Index**
   - Iterate through all documents
   - For each term, record which documents contain it
   - Store as dictionary with sets for fast lookups

3. **Query Processing**
   - O(1) lookup for single term
   - Set operations for Boolean queries
   - Much faster than scanning documents

4. **Performance**
   - Critical for large collections
   - Trades space for speed
   - Scales to billions of documents (Google, etc.)

### Index Statistics:
- **Space Overhead**: Stores only non-zero entries (sparse)
- **Build Time**: Linear in collection size, O(n)
- **Query Time**: O(1) for term lookup, O(k) for result merging

### Real-World Extensions:
1. **Positional Index**: Store term positions for phrase queries
2. **Term Frequency**: Store how many times term appears
3. **Compression**: Reduce index size (variable byte encoding)
4. **Distributed**: Shard index across multiple machines

### Next Steps:
In the next notebook (`05_vector_space_model.ipynb`), we will:
- Move beyond binary retrieval
- Represent documents as vectors
- Calculate similarity scores
- Rank documents by relevance

### Research References:
- Manning et al., "Introduction to Information Retrieval", Chapters 1-2
- Inverted index is used in: Lucene, Elasticsearch, Solr, Google, Bing
- First described by Gerard Salton in the 1960s